In [6]:
import os
import time
import pandas as pd
import numpy as np

import rasterio

import dask_gateway
import dask.array as da

from joblib import load

import raster_to_features as rm

In [11]:
#scene_ids = pd.read_csv(os.path.join(os.getcwd(),'temp','coastal_scenes_ids_2020.csv')).itemid.loc[19:26]
scene_ids = ['ca_m_3411934_sw_11_1_20140601_20141030']

In [3]:
# open pre-trained random forest classifier
rfc = load('spectral_rfc.joblib') 
print('loaded model')

loaded model


In [4]:
# initialize DASK cluster
cluster = dask_gateway.GatewayCluster()
cluster.scale(30)

client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.b75803c0821340b487c7320f75fb70eb/status,


In [12]:
for itemid in scene_ids:

    # ---------------------------------------
    # open NAIP scene
    t_alpha = time.time()
    raster = rm.rioxr_from_itemid(itemid)

    rm.raster_as_df(raster.to_numpy(), ['r','g','b','nir'])

    # find vegetation pixels to go into model
    # keep ndices of water and low-ndvi pixels
    # adds ndvi and ndwi features for each pixel
    t0 = time.time()
    is_veg, water_index, not_veg_index = rm.add_spectral_features(df = rm.raster_as_df(raster.to_numpy(), 
                                                                                       ['r','g','b','nir']), #names of bands
                                                               ndwi_thresh = 0.3, 
                                                               ndvi_thresh = 0.05) 
    # select features
    is_veg.drop('ndwi', axis=1, inplace=True)
    is_veg = rm.add_date_features(is_veg, raster.datetime)
    print('assembled pixels dataframe with features\n   time taken to assemble: ', time.time() - t0,' s')

    # ---------------------------------------
    # convert into dask.array and predict using model
    da_pixels = da.from_array(np.array(is_veg), chunks=728802)

    scene_preds = rfc.predict(da_pixels)    

    t0 = time.time()
    preds = scene_preds.compute()
    print('finished predicting\n   time taken to predict: ', time.time() - t0,' s')

    # ---------------------------------------
    # recover pixel indices for iceplant classifications
    preds_df = pd.DataFrame(preds, 
                         columns=['is_iceplant'], 
                         index = is_veg.index)
    is_iceplant_index = preds_df[preds_df.is_iceplant == 1].index.to_numpy()
    non_iceplant_index = preds_df[preds_df.is_iceplant == 0].index.to_numpy()

    # ---------------------------------------
    # reconstruct indices into image
    indices = [non_iceplant_index,
               is_iceplant_index, 
               not_veg_index,
               water_index]
    values = [0,    # values assigned to pixels from each index
              1,
              2,
              3]
    t0 = time.time()
    reconstruct = rm.indices_to_image(raster.shape[1], raster.shape[2], indices, values, back_value=100)
    print('reconstructed image\n   time taken to reconstruct: ', time.time() - t0,' s')

    # ---------------------------------------
    # save raster 
    filename = 'S_preds_' + itemid +'.tif'

    with rasterio.open(
        os.path.join(os.getcwd(),'temp','campus_point',filename),  # file path
        'w',           # w = write
        driver = 'GTiff', # format
        height = reconstruct.shape[0], 
        width = reconstruct.shape[1],
        count = 1,  # number of raster bands in the dataset
        dtype = rasterio.uint8,
        crs = raster.rio.crs,
        transform = raster.rio.transform(),
    ) as dst:
        dst.write(reconstruct.astype(rasterio.uint8), 1)
    # ---------------------------------------
    print('total tme:', time.time() - t_alpha)
    print('FINISHED: ', itemid , '\n')

assembled pixels dataframe with features
   time taken to assemble:  3.8543801307678223  s
finished predicting
   time taken to predict:  26.041579961776733  s
reconstructed image
   time taken to reconstruct:  1.6594209671020508  s
total tme: 36.66961669921875
FINISHED:  ca_m_3411934_sw_11_1_20140601_20141030 

